# Tutorial for one-photon imaging dataset.

This tutorial shows how to access the *Two Photon dataset* using `pynwb`. 

This dataset contains the 2-photon calcium imaging holographic stimulation data and visual stimuli events.

Contents:

- [Reading an NWB file](#read-nwb)
- [Access subject and task metadata](#access-subject)
- [Access Mesoscopic Imaging](#access-1p-imaging)
- [Access Two-photon Imaging](#access-2p-imaging)
- [Access TTL Signals](#ttl-signals)
- [Access Wheel Signal](#wheel-signal)
- [Access Raw Behavior](#raw-behavior)
- [Access Processed Behavior](#processed-behavior)
- [Access Visual Stimulus](#access-visualstim)

<!-- A schematic representation where the source data is saved in NWB:

![Alt text](./conversion_outline_diagram.png) -->

# Reading an NWB file <a name="read-nwb"></a>

This section demonstrates how to read an NWB file using `pynwb`.

Based on the [NWB File Basics](https://pynwb.readthedocs.io/en/stable/tutorials/general/plot_file.html#sphx-glr-tutorials-general-plot-file-py) tutorial from [PyNWB](https://pynwb.readthedocs.io/en/stable/#).

An [NWBFile](https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.NWBFile) represents a single session of an experiment. Each NWBFile must have a `session description`, `identifier`, and `session start time`.

Reading is carried out using the [NWBHDF5IO](https://pynwb.readthedocs.io/en/stable/pynwb.html#pynwb.NWBHDF5IO) class. To read the NWB file use the read mode ("r") to retrieve an NWBFile object.


In [ ]:
from pynwb import NWBHDF5IO
from pathlib import Path

# Choose which epoch to explore
session_id = '20201231_00002_dbvdual035'

# The file path to a .nwb file
root_path = Path("/media/amtra/Samsung_T5/CN_data")
output_dir_path = root_path / "Higley-conversion_nwb/nwb_stub/"
nwbfile_path = output_dir_path / f"{session_id}.nwb"
io = NWBHDF5IO(path=nwbfile_path, mode="r", load_namespaces=True)
nwbfile = io.read()

nwbfile

Importantly, the `session start time` is the reference time for all timestamps in the file. For instance, an event with a timestamp of 0 in the file means the event occurred exactly at the session start time.

The `session_start_time` is extracted from the ScanImage metadata (`epoch`) of the first .tiff of the epoch.

In [ ]:
nwbfile.session_start_time

# Access subject metadata <a name="access-subject"></a>

This section demonstrates how to access the [Subject](https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.Subject) field in an NWB file.

The [Subject](https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.Subject) field can be accessed as `nwbfile.subject`.


In [ ]:
nwbfile.subject

# Access Mesoscopic Imaging <a name="access-1p-imaging"></a>

This section demonstraces how to access the raw One Photon imaging data.

`NWB` organizes data into different groups depending on the type of data. Groups can be thought of as folders within the file. Here are some of the groups within an NWBFile and the types of data they are intended to store:

- `acquisition`: raw, acquired data that should never change
- `processing`: processed data, typically the results of preprocessing algorithms and could change

## Raw OnePhoton Imaging

The raw OnePhoton imaging data is stored in `pynwb.ophys.OnePhotonSeries` objects (for each channel and excitation type separately) which is added to `nwbfile.acquisition`. The data can be accessed as `nwbfile.acquisition['OnePhotonSeries_color_Excitation_color_Channel']`.

The data in [OnePhotonSeries](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.OnePhotonSeries) is stored as a three dimensional array: the first dimension is time (frame), the second and third dimensions represent x and y (width by height). 

In [ ]:
names_of_photon_series = nwbfile.acquisition.keys()
_ = [print(name) for name in names_of_photon_series if "OnePhoton" in name ]

In [ ]:
photon_series_blue_ex = nwbfile.acquisition[f"OnePhotonSeries"]
print(f"Rate: {photon_series_blue_ex.rate}")
print(f"Starting Time: {photon_series_blue_ex.starting_time}")
photon_series_blue_ex.imaging_plane


In [ ]:
photon_series_violet_ex = nwbfile.acquisition[f"OnePhotonSeriesIsosbestic"]
print(f"Rate: {photon_series_violet_ex.rate}")
print(f"Starting Time: {photon_series_violet_ex.starting_time}")
photon_series_violet_ex.imaging_plane

In [ ]:
from matplotlib import pyplot as plt

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(8, 3), dpi=300)

ax[0].imshow(photon_series_blue_ex.data[10].T, aspect="auto", cmap="RdYlBu_r")
ax[0].set_title(f"{photon_series_blue_ex.imaging_plane.excitation_lambda} Excitation")

ax[1].imshow(photon_series_violet_ex.data[10].T, aspect="auto", cmap="RdYlBu_r")
ax[1].set_title(f"{photon_series_violet_ex.imaging_plane.excitation_lambda} Excitation")

# Access Two-photon Imaging <a name="access-2p-imaging"></a>

The raw TwoPhoton imaging data is stored in `pynwb.ophys.TwoPhotonSeries` objects (for each channel and excitation type separately) which is added to `nwbfile.acquisition`. The data can be accessed as `nwbfile.acquisition['TwoPhotonSeries']`.

The data in [TwoPhotonSeries](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.TwoPhotonSeries) is stored as a three dimensional array: the first dimension is time (frame), the second and third dimensions represent x and y (width by height). 

In [ ]:
photon_series_green_ex = nwbfile.acquisition["TwoPhotonSeries"]
print(f"Rate: {photon_series_green_ex.rate}")
print(f"Starting Time: {photon_series_green_ex.starting_time}")
photon_series_green_ex.imaging_plane

In [ ]:
plt.imshow(photon_series_green_ex.data[10], aspect="auto", cmap="Greys_r")
plt.title(f"{photon_series_green_ex.imaging_plane.excitation_lambda} Excitation")

## Accessing the segmentation data

The segmentation output for the Two Photon Imaging data is stored in `nwbfile.processing["ophys"]`. 

In NWB, the [PlaneSegmentation](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.PlaneSegmentation) class stores the detected regions of interest in the [TwoPhotonSeries](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.TwoPhotonSeries) data. The [ImageSegmentation](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.ImageSegmentation) can contain multiple `PlaneSegmentation` tables, so that we can store results of different segmentation algorithms or different segmentation classes.

We can access the plane segmentation for the [TwoPhotonSeries](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.TwoPhotonSeries) data as 
`nwbfile.processing["ophys"]["ImageSegmentation"]["PlaneSegmentation"]`.


In [ ]:
plane_segmentation = nwbfile.processing["ophys"]["ImageSegmentation"]["PlaneSegmentation"][:]
plane_segmentation[:10]

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(8, 3), dpi=300)

ax[0].imshow(photon_series_green_ex.data[50], aspect="auto", cmap="RdYlBu_r")
ax[0].set_title("Raw image")

ax[1].imshow(plane_segmentation.image_mask[1], aspect="auto", cmap="RdYlBu_r")
ax[1].set_title("Image mask (single ROI)")

The summary images of the segmentation are stored in [Images](https://pynwb.readthedocs.io/en/stable/pynwb.base.html#pynwb.base.Images) container in NWB. 

In [ ]:
images = nwbfile.processing["ophys"]["SegmentationImages"]
images

In [ ]:
plt.imshow(photon_series_green_ex.data[50].T, cmap="Greys_r")
plt.title("TwoPhotonSeries")
plt.show()

plt.imshow(images.images["correlation"].data[:].T, cmap="RdYlBu_r")
plt.title("Image Correlation")
plt.show()

plt.imshow(images.images["mean"].data[:].T, cmap="RdYlBu_r")
plt.title("Image mean")
plt.show()

The fluroscence traces are stored in a [Fluorescence](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.Fluorescence) container, the raw traces can be accessed as `nwbfile.processing["ophys"]["Fluorescence"]["RoiResponseSeries"]`.

In [ ]:
nwbfile.processing["ophys"]["Fluorescence"]

## Visualize raw traces

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

data = nwbfile.processing["ophys"]["Fluorescence"][f"RoiResponseSeries"].data[:, :10]
rate = nwbfile.processing["ophys"]["Fluorescence"][f"RoiResponseSeries"].rate
df = pd.DataFrame(data)
df["time"] = np.linspace(0, data.shape[0]*rate,data.shape[0])
df.set_index("time", inplace=True)
df.columns.name = 'ROIs'

import plotly.express as px

fig = px.line(df, facet_row="ROIs", facet_row_spacing=0.01)

# hide and lock down axes
fig.update_xaxes(visible=True, fixedrange=False)
fig.update_yaxes(visible=False, fixedrange=False)

# remove facet/subplot labels
fig.update_layout(annotations=[], overwrite=True)

# strip down the rest of the plot
fig.update_layout(
    showlegend=True,
    plot_bgcolor="white",
    margin=dict(t=10, l=10, b=10, r=10)
)

fig.show(config=dict(displayModeBar=True))

In [ ]:
roi_index = 10
if plane_segmentation["Accepted"].values[roi_index]:
    description = "(accepted)"
else:
    description = "(rejected)"

import matplotlib.pyplot as plt

fig = plt.figure(figsize=(20,5))
data_roi_trace = nwbfile.processing["ophys"]["Fluorescence"]["RoiResponseSeries"].data[:, roi_index]
data_neuropil_trace = nwbfile.processing["ophys"]["Fluorescence"]["Neuropil"].data[:, roi_index]
rate = nwbfile.processing["ophys"]["Fluorescence"]["RoiResponseSeries"].rate
time = np.linspace(0, data.shape[0]*rate,data.shape[0])
plt.plot(time, data_roi_trace, label="activity")
plt.plot(time, data_neuropil_trace, label="neuropil")
plt.xlabel("Time (s)")
plt.ylabel("Df/f")
plt.title(f"ROI {roi_index} {description}")
plt.legend()


# Access TTL Signals <a name="#ttl-signals"></a>

This section demonstrates how to access the TTL signals stored in the NWBFile.
TTLTypesTable contains the description and the id for each TTL signal
TTLsTable contains the respective timestamps (rising times)

In [ ]:
import pandas as pd

events = pd.merge(
    left=nwbfile.acquisition["TTLsTable"][:],
    right=nwbfile.acquisition["TTLTypesTable"][:],
    left_on="ttl_type",
    right_on="id",
)
events.sort_values(by="ttl_type")
events.head()

# Access Wheel Signal <a name="#wheel-signal"></a>

This section demonstrates how to access the wheel velocity trace stored in the NWBFile.

In [ ]:
wheel_signal = nwbfile.acquisition["WheelSignal"]
wheel_signal

In [ ]:
import numpy as np
timestamps = wheel_signal.get_timestamps()
plt.plot(timestamps,wheel_signal.data*wheel_signal.conversion)
plt.ylabel(wheel_signal.unit)
plt.ylim(0,5)
plt.xlabel("Time (s)")
plt.title(wheel_signal.name)

# Access Raw Behavior <a name="#raw-behavior"></a>

This section demonstrates how to access the raw behavioral video stored in the NWBFile.

In [ ]:
video = nwbfile.acquisition[f"Video: {session_id}"]
video

# Access Processed Behavior <a name="#processed-behavior"></a>

This section demonstrates how to access the Facemap output stored in the NWBFile.


In [ ]:
nwbfile.processing["behavior"]["EyeTracking"]

In [ ]:
nwbfile.processing["behavior"]["PupilTracking"]

In [ ]:
motion_svd_series = nwbfile.processing["behavior"]["MotionSVDSeriesROI1"]
motion_svd_series

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

# Prepare data for plotting
pupil_area = nwbfile.processing["behavior"]["PupilTracking"]["pupil_area"]
timestamps = pupil_area.get_timestamps()

fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(8, 3), dpi=300, sharex=True)

# Plot data
ax[0].plot(timestamps[:500], pupil_area.data[:500,0], color="black", linewidth=0.8, label='Pupil Area')

# Plot control data with offset
ax[1].plot(timestamps[:500], motion_svd_series.data[:500,0], color="grey", linewidth=0.5, label='Face Motion')

ax[0].spines['top'].set_visible(False)
ax[0].spines['right'].set_visible(False)
ax[0].spines['bottom'].set_visible(False)
ax[1].spines['top'].set_visible(False)
ax[1].spines['right'].set_visible(False)

ax[0].legend(["Pupil Area"], frameon=False, bbox_to_anchor=(.95, 1), loc='upper left', prop={'size': 8})
ax[1].legend(["Face Motion"], frameon=False, bbox_to_anchor=(.96, 1), loc='upper left', prop={'size': 8})
ax[1].tick_params(axis='y', labelsize=8)
ax[0].tick_params(axis='y', labelsize=8)


plt.xlabel('Time (s)', fontsize=8)
plt.tick_params(axis='x', labelsize=8)
plt.show()

In [ ]:
motion_svd_masks = nwbfile.processing["behavior"]["MotionSVDMasksROI1"]
motion_svd_masks

In [ ]:
motion_svd_masks.mask_coordinates[:]

In [ ]:
motion_svd_masks.processed_frame_dimension[:]

In [ ]:
mask = motion_svd_masks.image_mask_index[0]
plt.imshow(mask, aspect="auto", cmap="RdYlBu_r" )
plt.title("First component mask")
plt.show()

# Access Visual Stimulus <a name="access-visualstim"></a>

This section demonstrates how to access the visual stimulus data.

In [ ]:
visual_stimulus_table = nwbfile.intervals["VisualStimulus"].to_dataframe()
visual_stimulus_table.head()